## Deliverable 2. Create a Customer Travel Destinations Map.

In [19]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [20]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Boca Do Acre,BR,-8.7522,-67.3978,73.18,97,65,1.70,broken clouds
1,1,Nanortalik,GL,60.1432,-45.2371,36.70,77,100,4.59,overcast clouds
2,2,Albany,US,42.6001,-73.9662,36.00,58,40,1.99,scattered clouds
3,3,Kapaa,US,22.0752,-159.3190,78.01,83,90,9.22,moderate rain
4,4,Veraval,IN,20.9000,70.3667,77.86,78,100,7.90,overcast clouds


In [21]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [22]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Kapaa,US,22.0752,-159.3190,78.01,83,90,9.22,moderate rain
4,4,Veraval,IN,20.9000,70.3667,77.86,78,100,7.90,overcast clouds
9,9,Laguna,US,38.4210,-121.4238,75.00,35,1,12.66,clear sky
11,11,Victoria,HK,22.2855,114.1577,77.00,78,0,1.01,clear sky
13,13,Nabire,ID,-3.3667,135.4833,82.67,79,89,2.19,overcast clouds
15,15,Yaan,NG,7.3833,8.5667,80.60,72,49,9.93,light rain
20,20,Aneho,TG,6.2333,1.6000,84.20,79,20,9.22,light rain
23,23,Rikitea,PF,-23.1203,-134.9692,77.58,74,100,11.99,light rain
26,26,Saint-Philippe,RE,-21.3585,55.7679,77.00,78,75,9.22,broken clouds
28,28,Tual,ID,-5.6667,132.7500,83.14,70,9,2.55,clear sky


In [23]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 206 entries, 3 to 698
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              206 non-null    int64  
 1   City                 206 non-null    object 
 2   Country              206 non-null    object 
 3   Lat                  206 non-null    float64
 4   Lng                  206 non-null    float64
 5   Max Temp             206 non-null    float64
 6   Humidity             206 non-null    int64  
 7   Cloudiness           206 non-null    int64  
 8   Wind Speed           206 non-null    float64
 9   Current Description  206 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 17.7+ KB


In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,78.01,moderate rain,22.0752,-159.3190,
4,Veraval,IN,77.86,overcast clouds,20.9000,70.3667,
9,Laguna,US,75.00,clear sky,38.4210,-121.4238,
11,Victoria,HK,77.00,clear sky,22.2855,114.1577,
13,Nabire,ID,82.67,overcast clouds,-3.3667,135.4833,
15,Yaan,NG,80.60,light rain,7.3833,8.5667,
20,Aneho,TG,84.20,light rain,6.2333,1.6000,
23,Rikitea,PF,77.58,light rain,-23.1203,-134.9692,
26,Saint-Philippe,RE,77.00,broken clouds,-21.3585,55.7679,
28,Tual,ID,83.14,clear sky,-5.6667,132.7500,


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,78.01,moderate rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
4,Veraval,IN,77.86,overcast clouds,20.9000,70.3667,Lords Inn Somnath
9,Laguna,US,75.00,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
11,Victoria,HK,77.00,clear sky,22.2855,114.1577,Mini Hotel Central
13,Nabire,ID,82.67,overcast clouds,-3.3667,135.4833,Hotel Nusantara Nabire
15,Yaan,NG,80.60,light rain,7.3833,8.5667,
20,Aneho,TG,84.20,light rain,6.2333,1.6000,Hotel La Linette
23,Rikitea,PF,77.58,light rain,-23.1203,-134.9692,Pension Maro'i
26,Saint-Philippe,RE,77.00,broken clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
28,Tual,ID,83.14,clear sky,-5.6667,132.7500,Grand Vilia Hotel


In [50]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)



hotel_df.dropna(subset=['Hotel Name'], inplace=True)


hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,78.01,moderate rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
4,Veraval,IN,77.86,overcast clouds,20.9000,70.3667,Lords Inn Somnath
9,Laguna,US,75.00,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
11,Victoria,HK,77.00,clear sky,22.2855,114.1577,Mini Hotel Central
13,Nabire,ID,82.67,overcast clouds,-3.3667,135.4833,Hotel Nusantara Nabire
20,Aneho,TG,84.20,light rain,6.2333,1.6000,Hotel La Linette
23,Rikitea,PF,77.58,light rain,-23.1203,-134.9692,Pension Maro'i
26,Saint-Philippe,RE,77.00,broken clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
28,Tual,ID,83.14,clear sky,-5.6667,132.7500,Grand Vilia Hotel
29,Port Blair,IN,77.00,scattered clouds,11.6667,92.7500,Welcomhotel Bay Island Port Blair


In [51]:
clean_hotel_df = hotel_df
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193 entries, 3 to 698
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 193 non-null    object 
 1   Country              193 non-null    object 
 2   Max Temp             193 non-null    float64
 3   Current Description  193 non-null    object 
 4   Lat                  193 non-null    float64
 5   Lng                  193 non-null    float64
 6   Hotel Name           193 non-null    object 
dtypes: float64(3), object(4)
memory usage: 12.1+ KB


In [54]:
# 8a. Create the output File (CSV)
output_data_file="../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [57]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [58]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))